<a href="https://colab.research.google.com/github/changsin/AI/blob/main/06.3.astar-8-puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving 8-puzzle using A* Search Algorithm

https://gist.github.com/flatline/838202


In [ ]:
import random
import math

_goal_state = [[1,2,3],
               [4,5,6],
               [7,8,0]]

def index(item, seq):
    """Helper function that returns -1 for non-found index value of a seq"""
    if item in seq:
        return seq.index(item)
    else:
        return -1

In [ ]:
class EightPuzzle:

    def __init__(self):
        # heuristic value
        self._hval = 0
        # search depth of current instance
        self._depth = 0
        # parent node in search path
        self._parent = None
        self.adj_matrix = []

        for i in range(3):
            self.adj_matrix.append(_goal_state[i][:])

    def __eq__(self, other):
        if self.__class__ != other.__class__:
            return False
        else:
            return self.adj_matrix == other.adj_matrix

    def __str__(self):
        res = ''
        for row in range(3):
            res += ' '.join(map(str, self.adj_matrix[row]))
            res += '\r\n'
        return res

    def _clone(self):
        p = EightPuzzle()
        for i in range(3):
            p.adj_matrix[i] = self.adj_matrix[i][:]
        return p

    def _get_legal_moves(self):
        """Returns list of tuples with which the free space may
        be swapped"""
        # get row and column of the empty piece
        row, col = self.find(0)
        free = []

        # find which pieces can move there
        if row > 0:
            free.append((row - 1, col))
        if col > 0:
            free.append((row, col - 1))
        if row < 2:
            free.append((row + 1, col))
        if col < 2:
            free.append((row, col + 1))

        return free

    def _generate_moves(self):
        free = self._get_legal_moves()
        zero = self.find(0)

        def swap_and_clone(a, b):
            p = self._clone()
            p.swap(a,b)
            p._depth = self._depth + 1
            p._parent = self
            return p

        return map(lambda pair: swap_and_clone(zero, pair), free)

    def _generate_solution_path(self, path):
        if self._parent == None:
            return path
        else:
            path.append(self)
            return self._parent._generate_solution_path(path)

    def set_board(self, start_str):
      self.adj_matrix = []

      id = 0
      for row in range(3):
        vector = []
        for col in range(3):
          ch = start_str[row * 3 + col]
          vector.append(int(ch))

        self.adj_matrix.append(vector)

    def solve(self, h):
        """Performs A* search for goal state.
        h(puzzle) - heuristic function, returns an integer
        """
        def is_solved(puzzle):
            return puzzle.adj_matrix == _goal_state

        openl = [self]
        closedl = []
        move_count = 0
        while len(openl) > 0:
            x = openl.pop(0)
            move_count += 1
            if (is_solved(x)):
                if len(closedl) > 0:
                    return x._generate_solution_path([]), move_count
                else:
                    return [x]

            succ = x._generate_moves()
            idx_open = idx_closed = -1
            for move in succ:
                # have we already seen this node?
                idx_open = index(move, openl)
                idx_closed = index(move, closedl)
                hval = h(move)
                fval = hval + move._depth

                if idx_closed == -1 and idx_open == -1:
                    move._hval = hval
                    openl.append(move)
                elif idx_open > -1:
                    copy = openl[idx_open]
                    if fval < copy._hval + copy._depth:
                        # copy move's values over existing
                        copy._hval = hval
                        copy._parent = move._parent
                        copy._depth = move._depth
                elif idx_closed > -1:
                    copy = closedl[idx_closed]
                    if fval < copy._hval + copy._depth:
                        move._hval = hval
                        closedl.remove(copy)
                        openl.append(move)

            closedl.append(x)
            openl = sorted(openl, key=lambda p: p._hval + p._depth)

        # if finished state not found, return failure
        return [], 0

    def shuffle(self, step_count):
        for i in range(step_count):
            row, col = self.find(0)
            free = self._get_legal_moves()
            target = random.choice(free)
            self.swap((row, col), target)
            row, col = target

    def find(self, value):
        """returns the row, col coordinates of the specified value
           in the graph"""
        if value < 0 or value > 8:
            raise Exception("value out of range")

        for row in range(3):
            for col in range(3):
                if self.adj_matrix[row][col] == value:
                    return row, col

    def peek(self, row, col):
        """returns the value at the specified row and column"""
        return self.adj_matrix[row][col]

    def poke(self, row, col, value):
        """sets the value at the specified row and column"""
        self.adj_matrix[row][col] = value

    def swap(self, pos_a, pos_b):
        """swaps values at the specified coordinates"""
        temp = self.peek(*pos_a)
        self.poke(pos_a[0], pos_a[1], self.peek(*pos_b))
        self.poke(pos_b[0], pos_b[1], temp)

In [ ]:
def heur(puzzle, item_total_calc, total_calc):
    """
    Heuristic template that provides the current and target position for each number and the
    total function.

    Parameters:
    puzzle - the puzzle
    item_total_calc - takes 4 parameters: current row, target row, current col, target col.
    Returns int.
    total_calc - takes 1 parameter, the sum of item_total_calc over all entries, and returns int.
    This is the value of the heuristic function
    """
    t = 0
    for row in range(3):
        for col in range(3):
            val = puzzle.peek(row, col) - 1
            target_col = val % 3
            target_row = val / 3

            # account for 0 as blank
            if target_row < 0:
                target_row = 2

            t += item_total_calc(row, target_row, col, target_col)

    return total_calc(t)

#some heuristic functions, the best being the standard manhattan distance in this case, as it comes
#closest to maximizing the estimated distance while still being admissible.

def h_manhattan(puzzle):
    return heur(puzzle,
                lambda r, tr, c, tc: abs(tr - r) + abs(tc - c),
                lambda t : t)

def h_manhattan_lsq(puzzle):
    return heur(puzzle,
                lambda r, tr, c, tc: (abs(tr - r) + abs(tc - c))**2,
                lambda t: math.sqrt(t))

def h_linear(puzzle):
    return heur(puzzle,
                lambda r, tr, c, tc: math.sqrt(math.sqrt((tr - r)**2 + (tc - c)**2)),
                lambda t: t)

def h_linear_lsq(puzzle):
    return heur(puzzle,
                lambda r, tr, c, tc: (tr - r)**2 + (tc - c)**2,
                lambda t: math.sqrt(t))

def h_default(puzzle):
    return 0

def main(start_str):

    p = EightPuzzle()
    print(p)
    if start_str is None:
      p.shuffle(20)
    else:
      p.set_board(start_str)
    print(p)

    path, count = p.solve(h_manhattan)
    path.reverse()
    for i in path:
        print(i)

    print("Solved with Manhattan distance exploring", count, "states")
    path, count = p.solve(h_manhattan_lsq)
    print("Solved with Manhattan least squares exploring", count, "states")
    path, count = p.solve(h_linear)
    print("Solved with linear distance exploring", count, "states")
    path, count = p.solve(h_linear_lsq)
    print("Solved with linear least squares exploring", count, "states")
#    path, count = p.solve(heur_default)
#    print "Solved with BFS-equivalent in", count, "moves"

main("134702865")

1 2 3
4 5 6
7 8 0

1 3 4
7 0 2
8 6 5

1 3 4
7 2 0
8 6 5

1 3 0
7 2 4
8 6 5

1 0 3
7 2 4
8 6 5

1 2 3
7 0 4
8 6 5

1 2 3
7 4 0
8 6 5

1 2 3
7 4 5
8 6 0

1 2 3
7 4 5
8 0 6

1 2 3
7 4 5
0 8 6

1 2 3
0 4 5
7 8 6

1 2 3
4 0 5
7 8 6

1 2 3
4 5 0
7 8 6

1 2 3
4 5 6
7 8 0

Solved with Manhattan distance exploring 35 states
Solved with Manhattan least squares exploring 257 states
Solved with linear distance exploring 119 states
Solved with linear least squares exploring 419 states


# Uninformed Search: BFS, DFS, and IDDFS

## Simple 8-Puzzle Class

This is a simpler implementation of the 8-puzzle class with basic functionalities required for search algorithms.

In [1]:
import random

class SimpleEightPuzzle:
    def __init__(self, board=None, parent=None, move=None):
        if board is None:
            # Initialize with the goal state and shuffle
            self.board = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
            self.shuffle(20) # Shuffle to create a solvable puzzle
        else:
            self.board = board
        self.parent = parent
        self.move = move  # The move that led to this state
        self.depth = 0 if parent is None else parent.depth + 1

    def __eq__(self, other):
        if not isinstance(other, SimpleEightPuzzle):
            return False
        return self.board == other.board

    def __hash__(self):
        # Generate a hash based on the flattened board tuple for efficient set lookups
        return hash(tuple(sum(self.board, [])))

    def __str__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.board])

    def get_blank_position(self):
        """Finds the row and column of the blank space (0)."""
        for r in range(3):
            for c in range(3):
                if self.board[r][c] == 0:
                    return r, c
        return -1, -1

    def get_possible_moves(self):
        """Returns a list of possible moves (row, col delta)."""
        row, col = self.get_blank_position()
        moves = []
        if row > 0: moves.append((-1, 0))  # Up
        if row < 2: moves.append((1, 0))   # Down
        if col > 0: moves.append((0, -1))  # Left
        if col < 2: moves.append((0, 1))   # Right
        return moves

    def apply_move(self, move):
        """Applies a move to the current board and returns the new board state."""
        new_board = [row[:] for row in self.board] # Create a copy
        blank_row, blank_col = self.get_blank_position()
        move_row, move_col = move
        target_row, target_col = blank_row + move_row, blank_col + move_col

        # Swap the blank space with the target tile
        new_board[blank_row][blank_col] = new_board[target_row][target_col]
        new_board[target_row][target_col] = 0
        return new_board


    def get_successors(self):
        """Generates successor states from the current state."""
        successors = []
        possible_moves = self.get_possible_moves()
        for move in possible_moves:
            new_board = self.apply_move(move)
            successors.append(SimpleEightPuzzle(new_board, self, move))
        return successors

    def shuffle(self, step_count):
        """Shuffles the puzzle from the goal state."""
        current_board = [row[:] for row in self.board]
        for _ in range(step_count):
            # Find blank position in the current board state
            blank_row, blank_col = -1, -1
            for r in range(3):
                for c in range(3):
                    if current_board[r][c] == 0:
                        blank_row, blank_col = r, c
                        break
                if blank_row != -1: break

            possible_moves = []
            if blank_row > 0: possible_moves.append((-1, 0))
            if blank_row < 2: possible_moves.append((1, 0))
            if blank_col > 0: possible_moves.append((0, -1))
            if blank_col < 2: possible_moves.append((0, 1))

            if not possible_moves: continue # Should not happen for a valid puzzle

            move = random.choice(possible_moves)
            move_row, move_col = move
            target_row, target_col = blank_row + move_row, blank_col + move_col

            # Swap the blank space with the target tile in the current board state
            current_board[blank_row][blank_col] = current_board[target_row][target_col]
            current_board[target_row][target_col] = 0

        self.board = current_board


    def set_board(self, start_str):
        """Sets the board from a string representation."""
        self.board = []
        row = []
        for i, char in enumerate(start_str):
            row.append(int(char))
            if (i + 1) % 3 == 0:
                self.board.append(row)
                row = []

    def get_solution_path(self):
        """Returns the path from the initial state to the current state."""
        path = []
        current = self
        while current:
            path.append(current)
            current = current.parent
        path.reverse()
        return path

## Breadth-First Search (BFS) - Simple Implementation

Here is the simple iterative implementation of BFS using the `SimpleEightPuzzle` class.

In [5]:
from collections import deque

def bfs_simple(start_node):
    """
    Performs Breadth-First Search for the goal state.

    Parameters:
    start_node - the initial state of the puzzle
    """
    goal_board = [[1, 2, 3], [4, 5, 6], [7, 8, 0]] # Define goal_board here

    def is_solved(puzzle):
        return puzzle.board == goal_board

    openl = deque([start_node])  # Use a deque for efficient FIFO operations
    visited = set()
    visited.add(start_node)
    move_count = 0

    while openl:
        x = openl.popleft()  # Use popleft() for BFS (FIFO)
        move_count += 1

        if is_solved(x):
            return x.get_solution_path(), move_count

        successors = x.get_successors()
        for successor in successors:
            if successor not in visited:
                visited.add(successor)
                openl.append(successor)

    # If finished state not found, return failure
    return [], 0

# Test BFS Simple
print("Solving with Simple BFS:")
p_bfs_simple = SimpleEightPuzzle()
# Use a solvable starting configuration
p_bfs_simple.set_board("134702865")
path_bfs_simple, count_bfs_simple = bfs_simple(p_bfs_simple)
if path_bfs_simple:
    for i in path_bfs_simple:
        print(i)
        print("-" * 10)
    print("Solved with Simple BFS exploring", count_bfs_simple, "states")
else:
    print("Simple BFS could not find a solution.")

Solving with Simple BFS:
1 3 4
7 0 2
8 6 5
----------
1 3 4
7 2 0
8 6 5
----------
1 3 0
7 2 4
8 6 5
----------
1 0 3
7 2 4
8 6 5
----------
1 2 3
7 0 4
8 6 5
----------
1 2 3
7 4 0
8 6 5
----------
1 2 3
7 4 5
8 6 0
----------
1 2 3
7 4 5
8 0 6
----------
1 2 3
7 4 5
0 8 6
----------
1 2 3
0 4 5
7 8 6
----------
1 2 3
4 0 5
7 8 6
----------
1 2 3
4 5 0
7 8 6
----------
1 2 3
4 5 6
7 8 0
----------
Solved with Simple BFS exploring 2222 states


## Depth-First Search (DFS) - Simple Implementation

Here is the simple recursive implementation of DFS using the `SimpleEightPuzzle` class.

In [ ]:
def is_solved(puzzle):
    """Checks if the current puzzle state is the goal state."""
    return puzzle.board == [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

def dfs_iterative_simple(start_node):
    """
    Performs iterative Depth-First Search for the goal state.

    Parameters:
    start_node - the initial state of the puzzle
    """
    openl = [start_node]  # Use a list as a stack (LIFO)
    visited = set()
    visited.add(start_node)
    move_count = 0

    while openl:
        x = openl.pop()  # Get the last element (LIFO)
        move_count += 1

        if is_solved(x):
            return x.get_solution_path(), move_count

        successors = x.get_successors()
        # Add successors to the stack in reverse order to explore the first successor first
        for successor in reversed(successors):
            if successor not in visited:
                visited.add(successor)
                openl.append(successor)

    # If finished state not found, return failure
    return [], 0

# Test DFS Simple (Iterative)
print("Solving with Simple DFS (Iterative):")
p_dfs_simple = SimpleEightPuzzle()
# Use a solvable starting configuration
p_dfs_simple.set_board("134702865")
path_dfs_simple, count_dfs_simple = dfs_iterative_simple(p_dfs_simple)
if path_dfs_simple:
    for i in path_dfs_simple:
        print(i)
        print("-" * 10)
    print("Solved with Simple DFS (Iterative) exploring", count_dfs_simple, "states")
else:
    print("Simple DFS (Iterative) could not find a solution.")

## Iterative Deepening Depth-First Search (IDDFS)

Iterative Deepening Depth-First Search (IDDFS) is a state space search algorithm that combines depth-first search's space-efficiency and breadth-first search's completeness. It does this by performing a series of depth-limited depth-first searches with increasing depth limits.

In [3]:
def dls_recursive(current_node, limit, move_count_ref):
    """
    Performs recursive Depth-Limited Search.

    Parameters:
    current_node - the current state of the puzzle
    limit - the current depth limit
    move_count_ref - a list containing a single element to track the number of moves
    """
    move_count_ref[0] += 1

    if is_solved(current_node):
        return current_node.get_solution_path()

    if limit == 0:
        return None # Reached depth limit

    successors = current_node.get_successors()
    for successor in successors:
        path = dls_recursive(successor, limit - 1, move_count_ref)
        if path:
            return path

    return None

def iddfs(start_node):
    """
    Performs Iterative Deepening Depth-First Search.

    Parameters:
    start_node - the initial state of the puzzle
    """
    move_count = 0
    for depth_limit in range(100): # Set a reasonable maximum depth limit
        move_count_ref = [0]
        path = dls_recursive(start_node, depth_limit, move_count_ref)
        move_count += move_count_ref[0]
        if path:
            return path, move_count
    return [], move_count # No solution found within the depth limit

# Test IDDFS
print("Solving with IDDFS:")
p_iddfs = SimpleEightPuzzle()
# Use a solvable starting configuration
p_iddfs.set_board("134702865")
path_iddfs, count_iddfs = iddfs(p_iddfs)
if path_iddfs:
    for i in path_iddfs:
        print(i)
        print("-" * 10)
    print("Solved with IDDFS exploring", count_iddfs, "states")
else:
    print("IDDFS could not find a solution within the depth limit.")

Solving with IDDFS:
1 3 4
7 0 2
8 6 5
----------
1 3 4
7 2 0
8 6 5
----------
1 3 0
7 2 4
8 6 5
----------
1 0 3
7 2 4
8 6 5
----------
1 2 3
7 0 4
8 6 5
----------
1 2 3
7 4 0
8 6 5
----------
1 2 3
7 4 5
8 6 0
----------
1 2 3
7 4 5
8 0 6
----------
1 2 3
7 4 5
0 8 6
----------
1 2 3
0 4 5
7 8 6
----------
1 2 3
4 0 5
7 8 6
----------
1 2 3
4 5 0
7 8 6
----------
1 2 3
4 5 6
7 8 0
----------
Solved with IDDFS exploring 797174 states
